In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import networkx as nx

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

allgames = pd.read_csv("/kaggle/input/chess/games.csv")
print(allgames['white_id'].unique().size)
print(allgames['black_id'].unique().size)
all_players = np.append(allgames['white_id'].unique(), allgames['black_id'].unique())
print(all_players.size)
all_players = np.unique(all_players)
print(all_players.size)




# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
white_success = allgames.loc[(allgames.winner == 'white')].groupby('white_id').winner.count()
black_success = allgames.loc[(allgames.winner == 'black')].groupby('black_id').winner.count()

success_dict = {}
#
for i, v in white_success.items():
    success_dict[i] = v
#
for i, v in black_success.items():
    if i in success_dict.keys():
        success_dict[i] = int(v) + int(success_dict[i])
    else:
        success_dict[i] = int(v)
# print(success_dict)

In [ ]:
white_all = allgames.groupby('white_id').winner.count()
black_all = allgames.groupby('black_id').winner.count()
all_dict = {}
#
for i, v in white_all.items():
    all_dict[i] = v
#
for i, v in black_all.items():
    if i in all_dict.keys():
        all_dict[i] = int(v) + int(all_dict[i])
    else:
        all_dict[i] = int(v)
# print(all_dict)

In [ ]:
#并没有展示除 Power Law
success_list = sorted(success_dict.items(),key=lambda item: item[1],reverse=True)
# print(len(success_list))
# print(len(all_dict))
# # print(success_dict)
# print(type(success_list))
# print(type(success_list[0]))
# print(success_list[0])
x_list = [];
y_list = [];
for k_tuple in success_list:
    x_list.append(all_dict[k_tuple[0]]) #橫軸為參加比賽的次數
    y_list.append(k_tuple[1])#Y軸為獲勝的次數
x_narray = np.array(x_list)
y_narray = np.array(y_list)
plt.xlabel('Competition Candidation Times')
plt.ylabel('Competition Success Times')
plt.plot(x_narray,y_narray)


In [ ]:
person_success_num = pd.DataFrame(success_list,columns=['name','number'])
# print(person_success_num)
# print(type(person_success_num))
# print(person_success_num.groupby('number').count())
# person_success_num_summary = person_success_num.groupby('number').number.count()
person_success_num_summary = person_success_num.groupby('number').number.count()
# print(person_success_num_summary)
# print(type(person_success_num_summary))

x_list = []
y_list = []

# for index, row in person_success_num_summary.iterrows():
#     print (row["number"] + "," + row["name"])
for i, v in person_success_num_summary.items():
    x_list.append(i)
    y_list.append(v)
# print(x_list)
# print(y_list)
x_narray = np.array(x_list) 
y_narray = np.array(y_list)
plt.xlabel('Competition Success Times')
plt.ylabel('Relative Candidate Number')
plt.plot(x_narray,y_narray,'.-')



In [ ]:

all_dict = sorted(all_dict.items(),key=lambda item: item[1],reverse=True)
person_all_num = pd.DataFrame(all_dict,columns=['name','number'])
# print(person_success_num)
# print(type(person_success_num))
# print(person_success_num.groupby('number').count())
# person_success_num_summary = person_success_num.groupby('number').number.count()
person_all_num_summary = person_all_num.groupby('number').number.count()
# print(person_success_num_summary)
# print(type(person_success_num_summary))

x_list = []
y_list = []

# for index, row in person_success_num_summary.iterrows():
#     print (row["number"] + "," + row["name"])
for i, v in person_all_num_summary.items():
    x_list.append(i)
    y_list.append(v)
# print(x_list)
# print(y_list)
x_narray = np.array(x_list) #參賽次數
y_narray = np.array(y_list) #參賽人數
plt.xlabel('Competition Times')
plt.ylabel('Relative Candidate Number')
plt.plot(x_narray,y_narray,'.-')

In [ ]:
# def func(x, a, b, c):
#     return a * np.exp(-b * x) + c
# popt, pcov = curve_fit(func, x_narray, y_narray)
def func(x, A, Y):
    return A * np.power(x, -Y)
popt, pcov = curve_fit(func, x_narray, y_narray)
print(popt)
print(pcov)
y2_narray = [func(i, popt[0],popt[1]) for i in x_narray]
plt.plot(x_narray,y2_narray,'r--')

In [ ]:
plt.loglog(basex=10,basey=10)
# Axes.set_yscale("log", base=10)
# Axes.set_xscale("log", base=10)
plt.plot(x_narray,y_narray)

In [ ]:
g=nx.Graph()
for k_tuple in all_dict:
    g.add_node(k_tuple[0])

edge_data = allgames.loc[:,['white_id', 'black_id']]
for index, row in edge_data.iterrows():
#     print(row['white_id'], row['black_id'])
    g.add_edge(row['white_id'],row['black_id'])

# print("alll has added, creating graph.... please wait")
# nx.draw(g)
# print("alll has added, creating graph.... please wait")
# plt.show()